This is an example of how the package "continuum" can be used to create a grid classification using in The degree of urbanisation level 1, from "A recommendation on the method to delineate cities, urban and rural areas for international statistical comparisons"

The data can be download from https://landscan.ornl.gov/ or https://ghsl.jrc.ec.europa.eu/datasets.php#inline-nav-ghs_pop2022

In [16]:
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import rasterstats

from shapely.geometry import box
from pyproj import CRS

import sys
sys.path.append(r"...\\01. Continuo urbano-rural")

import continuum

In [4]:
path=r'...\\01. Dataset'

In [52]:
def box_32718(box_coords):
    bb=box(box_coords[0],box_coords[1],box_coords[2],box_coords[3])
    
    box_0=gpd.GeoDataFrame()
    box_0['geometry']=None
    box_0.at[0,'geometry']=bb
    
    box_0.crs = CRS.from_epsg(4326)
    return box_0

coor=[-80.60,-3.72,-79.88,-3.20]
area1=box_32718(coor)

##cortamos el raster 
with rxr.open_rasterio(path+"\\landscan-global-2017.tif", masked=True) as des_pop:
    ## using rasterio function clip and the polygons geometry, we create a new raster for Cajamarca Region
    clipped = des_pop.rio.clip(area1.geometry, area1.crs, drop=True, from_disk=True)
    ## save the clipped raster inside the data base folder in the local machine
    clipped.rio.to_raster(path+"\\north_border_clip.tif")

#### Calculamos los polígonos de densidad

In [53]:
pop_density=(path+"\\north_border_clip.tif")
band_1, aff_1=continuum.open_raster_rio(pop_density)

## Setting the values for each density's cluster
density_values=[1500, 300, 50]
population_values=[50000, 5000, 0]

## Calculate the high, medium and low density's cluster
high=(continuum.create_continuum(density_values[0], band=band_1, affine=aff_1, no_holes=True,crs_EPSG=4326, pixel_con=8).
      pipe(continuum.add_pop_sum,pop_density,population_values[0]))
medium=(continuum.create_continuum(density_values[1],band=band_1, affine=aff_1, no_holes=True,crs_EPSG=4326, pixel_con=8).
       pipe(continuum.add_pop_sum,pop_density,population_values[1]))
low=(continuum.create_continuum(density_values[2], band=band_1, affine=aff_1, crs_EPSG=4326).
    pipe(continuum.add_pop_sum,pop_density,population_values[2]))

In [54]:
## Create a new id
def new_id(gdf_d,var):
    gdf_f=(gdf_d.
           assign(id_n=gdf_d.
           sort_values(var,ascending=False).
           reset_index(drop=True).
           index+1))
    return gdf_f

high=new_id(high,"pop_t")
medium=new_id(medium,"pop_t")
low=new_id(low,"pop_t")

#### Calculamos los valores de la tipología

In [55]:
np_f=continuum.spatial_tipology(high, medium, low)

high=high.merge(np_f[np_f.density=="high"], on='id_n', validate='1:1')
medium=medium.merge(np_f[np_f.density=="medium"], on='id_n', validate='1:1')
low=low.merge(np_f[np_f.density=="low"], on='id_n', validate='1:1')

In [86]:
a=low.explore(color='#1b9e77',style_kwds={"opacity":0.3})
medium.explore(m=a,color='#7570b3',style_kwds={"stroke":False},)
high.explore(m=a, color='#d95f02', style_kwds={"opacity":0.3})